In [2]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
import os
from keras.engine import training_utils  
from sklearn.utils import class_weight
from sklearn import preprocessing

#Defining main model characteristics:

SEQ_LEN = 60  # how long of a preceeding sequence to collect 
FUTURE_PERIOD_PREDICT = 12  # how far into the future are we trying to predict
VALIDATION_SPLIT = 0.2

EPOCHS = 100  # how many passes through our data
BATCH_SIZE = 128  # 64

NNNAME = "DENSE"

NAME = f"{NNNAME}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

SAVE = True

# Create models folder to save data in
if not os.path.isdir('models'):
    os.makedirs('models')


Using TensorFlow backend.


In [5]:
# classify based on:
#   * midpoint of best ask & best bid  later vs now
#   * buy at best ask & sell at best bid
#   * buy at last buy price & sell at last sell price
#   * can make profitable trade from OB other side

# changable:
#   * threshold
#   * how far into the future to comparedef classify(df):

    change = 0.0015 # Safety factor in percentage
    for i, row in df.iterrows():
        target = 1
        if df.at[i, "future_best_bid"] > df.at[i, "best_ask"] * (1 + change): # good to buy now
            target = 2
        elif df.at[i, "future_best_ask"] * (1 + change) < df.at[i, "best_bid"]: # good to sell now  
            target = 0
        df.at[i,"target"] = target
        

The three classes are calculated based on the change in return after the a certain time passed, with a threshold value of 0.0015 pecent

In [6]:
# Preprocessing the dataset to a uniform output format
def preprocess(df, shorten=False):
    df = df.loc[:,~df.columns.str.startswith('future_')]
    
    sprd = 2 * (df["best_ask"] - df["best_bid"]) / (df["best_ask"] + df["best_bid"])
    df.insert(2, "spread", sprd)
    
    ob_maxsum = df.filter(regex="(a|b)\d").sum(axis=1).max()
    ob_maxamt = df.filter(regex="last_buy_amt|last_sell_amt").sum(axis=1).max()
    
    for col in df.columns:
        if col in ["target", "spread"]:  # normalize all except for target, spread already done
            continue
        
        if col in ["best_ask", "best_bid", "last_buy_price", "last_sell_price"]:
            df[col] = df[col].pct_change()
        elif col in ["last_buy_amt", "last_sell_amt"]:
            df[col] = df[col] / ob_maxamt # preprocessing.scale(df[col].values)
        else:
            df[col] = df[col]/ob_maxsum
            
        df.dropna(inplace=True)  # remove the nas created by pct_change
   
    
    df.dropna(inplace=True)  

    sequential_data = []  # this is a list that will CONTAIN the sequences
    # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in
    prev_days = deque(maxlen=SEQ_LEN)
    
    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])

    buys = []  # list that will store our buy sequences and targets
    sells = []  # list that will store our sell sequences and targets
    holds = []

    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 1:
            holds.append([seq, target])
        elif target == 2:
            buys.append([seq, target])
        elif target == 0:
            sells.append([seq, target])

    if shorten:
        # make sure both lists are only up to the shortest length.
        minlen = min(len(buys), len(sells), len(holds))
        buys = buys[:minlen]
        sells = sells[:minlen]
        holds = holds[:minlen]


    sequential_data = buys+sells+holds  # add them together
    # another shuffle, so the model doesn't get confused with all 1 class then the other.
    random.shuffle(sequential_data)

    X = []
    y = []

    for seq, target in sequential_data:
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels

    return np.array(X), y 



In [3]:
path = "/Users/Ranykhalil/Downloads/Archive_2/"

data_file = "obbinsandtrades_BTC-USD_2020-03-26-23.49.57.83_2020-04-03-15.49.47.83_17196840.csv"
 
def read_ob_csv(file):
    df = pd.read_csv(path + file)
    df.set_index("time", inplace=True)
    df_len = len(df.index.unique())
    df_interval = df.index.unique()[1] - df.index.unique()[0]
    df_init_t = df.index.unique()[0]
    df_bins = len(df.loc[df.index == df_init_t].columns) - 6
    return df_len, df_interval, df_init_t, df_bins, df

df_len, df_interval, df_init_t, df_bins, df = read_ob_csv(data_file)

df.fillna(method="ffill", inplace=True)
df.dropna(inplace=True)

df.head()

,best_ask,best_bid,last_buy_price,last_buy_amt,last_sell_price,last_sell_amt,a0,a1,a2,a3,...,b90,b91,b92,b93,b94,b95,b96,b97,b98,b99
time,,,,,,,,,,,,,,,,,,,,,
1585266597,6750.92,6748.04,6750.00,0.000006,6752.40,0.004770,6.383364,1.487042,7.997782,0.000000,...,0.003176,2.430000,0.000,0.000000,2.664000,0.924243,13.609199,0.006484,18.500000,0.000
1585266607,6749.51,6747.00,6745.51,0.006459,6750.32,0.005308,0.002629,0.515601,0.503189,10.170660,...,2.430000,0.000000,0.000,2.664000,0.924243,13.609199,0.006484,18.500000,0.000000,0.000
1585266617,6748.55,6746.32,6749.09,0.300000,6749.07,0.008967,17.083271,2.912589,0.741189,1.031293,...,2.355000,0.000000,0.000,3.568243,0.078010,13.556673,0.001000,18.500000,0.000000,0.000
1585266627,6748.47,6748.46,6748.46,0.239064,6748.47,0.116225,0.008405,0.989471,0.000000,0.003189,...,0.273176,0.075000,2.355,0.000000,0.000000,3.568243,0.078010,13.556673,0.001000,18.500
1585266637,6750.00,6749.99,6748.46,0.239064,6750.00,0.021671,3.113139,0.000000,8.730782,1.397993,...,0.222206,0.273176,0.000,2.430000,0.000000,0.000000,2.664000,0.982253,13.555673,0.002


In [5]:
if "future_best_ask" not in df.columns:
    for col in df.columns:
        df['future_'+col] = df[col].shift(-FUTURE_PERIOD_PREDICT)
    
classify(df)
df.dropna(inplace=True)
print("[ SELL  HOLD  BUY ]")
print(df.groupby("target").count()["best_ask"].to_list())

[ SELL  HOLD  BUY ]
[9056, 88077, 9415]


In [6]:
#Data split non randomised
times = sorted(df.index.values)
validation_times = sorted(df.index.values)[-int(VALIDATION_SPLIT*len(times))]

validation_df = df[(df.index >= validation_times)]
df = df[(df.index < validation_times)]

# Run data through preprocess function above
train_x, train_y = preprocess(df, False)
validation_x, validation_y = preprocess(validation_df, False)

print("shape tr x", np.shape(train_x))
print("shape tr y", np.shape(train_y))
print("shape val x", np.shape(validation_x))
print("shape val y", np.shape(validation_y))

shape tr x (85176, 60, 207)
shape tr y (85176,)
shape val x (21246, 60, 207)
shape val y (21246,)


The model we are looking to create in the following case is dense model with 5 layers with a final node count of three: hold, buy, sell

In [10]:
print(f"""###
### TRAIN      \t total: {len(train_x)} \t holds: {train_y.count(1)} \t buys: {train_y.count(2)} \t sells: {train_y.count(0)}
### VALIDATION \t total: {len(validation_x)} \t holds: {validation_y.count(1)} \t buys: {validation_y.count(2)} \t sells: {validation_y.count(0)}
###""")

print(train_x.shape)
print(train_x.shape[1:])

model = Sequential()
model.add(Flatten(input_shape=(train_x.shape[1:])))

n_cells = 256 # 128
do_layer  = 0.2

model.add(Dense(n_cells, activation='relu'))
model.add(Dropout(do_layer))
model.add(BatchNormalization())

model.add(Dense(n_cells, activation='relu'))
model.add(Dropout(do_layer))
model.add(BatchNormalization())

model.add(Dense(n_cells, activation='relu'))
model.add(Dropout(do_layer))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(3, activation='softmax'))


opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['sparse_categorical_accuracy']
)
# Saving the model ----------------------------
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME)) 
# unique file name that will include the epoch and the validation acc for that epoch
filepath = NNNAME + "-{epoch:02d}-{val_sparse_categorical_accuracy:.3f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_sparse_categorical_accuracy',
                                                   verbose=1, save_best_only=True, mode='max'))  # saves only the best ones
# -------------------------

# Find the balanced class weights
train_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
    class_weight=dict(enumerate(train_weights)),
)

###
### TRAIN      	 total: 85176 	 holds: 70239 	 buys: 7634 	 sells: 7303
### VALIDATION 	 total: 21246 	 holds: 17748 	 buys: 1774 	 sells: 1724
###
(85176, 60, 207)
(60, 207)
Train on 85176 samples, validate on 21246 samples
Epoch 1/100
85176/85176 [==============================] - 44s 521us/sample - loss: 1.1387 - sparse_categorical_accuracy: 0.4169 - val_loss: 0.9477 - val_sparse_categorical_accuracy: 0.6391
Epoch 2/100
85176/85176 [==============================] - 39s 461us/sample - loss: 0.9912 - sparse_categorical_accuracy: 0.4789 - val_loss: 1.1051 - val_sparse_categorical_accuracy: 0.3698
Epoch 3/100
85176/85176 [==============================] - 56s 656us/sample - loss: 0.8754 - sparse_categorical_accuracy: 0.5229 - val_loss: 0.9240 - val_sparse_categorical_accuracy: 0.5541
Epoch 4/100
85176/85176 [==============================] - 51s 593us/sample - loss: 0.7488 - sparse_categorical_accuracy: 0.5596 - val_loss: 1.1969 - val_sparse_categorical_accuracy: 0.4173
Epoch 5/100


After all the epoch runs the validation accuracy is found to be 0.7 while the train data was at 0.9 indicating an overfit model but even with this large difference a validation score of 0.7 is considered a relatively high number especially comparing it to the logistic regression of the baseline model that only included a positive, negative accuracy calculation found to be 0.54



Next, we will try looking at the model run through an LSTM (Long Short Term Memory) model. LSTM networks used to classify, process and make predictions based on time series data, since there can be lags of unknown duration between important events in a time series. 